# Recommendation System

In [212]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
# import libraries
import pandas as pd
import numpy as np
import re
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

import spacy
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import pickle
from sklearn.utils import shuffle

from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [3]:
# Create a blank Tokenizer with just English
nlp=English()
tokenizer = Tokenizer(nlp.vocab)

In [4]:
# Import data
post_data = pd.read_csv(r"Data\post_data.csv")
user_data = pd.read_csv(r"Data\user_data.csv")
view_data = pd.read_csv(r"Data\view_data.csv")

In [5]:
temp_df = pd.DataFrame(view_data)
# Make NumLikes for KNN
temp_df["NumLikes"] = np.random.randint(1, 11, len(temp_df))
display(temp_df.head())
display(temp_df.shape)

,user_id,post_id,time_stamp,NumLikes
0,5eece14ffc13ae660900008b,136781766,01/01/2019 01:30 PM,8
1,5eece14efc13ae660900003c,43094523,01/01/2019 01:33 PM,10
2,5eece14efc13ae6609000025,42428071,01/01/2019 01:43 PM,4
3,5eece14ffc13ae66090001d4,76472880,01/01/2019 01:54 PM,1
4,5eece14ffc13ae66090000ac,202721843,01/01/2019 02:00 PM,8


(71800, 4)

In [6]:
df = pd.merge(temp_df, post_data ,on='post_id')

In [7]:
# Create new more detailed post categories
df['category'] = df['category'].replace({'banking':'Mystery,Educational,Cyberpunk',
                                         'programming':'Fantasy,Magical-realism,Western',
                                         'drawing':'Sci-fi,Steamy,Alternate-history',
                                         'politics':'Romance,Young-adult,Adventure,Fantasy',
                                         'zoology':'Kids,Western,Inspirational,Mystery',
                                         'art':'Educational,Drama,Kids',
                                         'graphic':'Inspirational,Romance,Magical-realism',
                                         'Music':'Alternate-history,Adventure,Satire,Western',
                                         'Mathematics':'Cyberpunk,Steamy,Satire,Drama',
                                         'photography':'Magical realism,Adventure,Fantasy,Action',
                                         'GST':'Historical Fiction,Young-adult,Realistic-fiction,Drama',
                                         'political':'Realistic Fiction,Drama,Adventure,Satire',
                                         'business':'Horror,Action,Mystery,Cyberpunk',
                                         'dance':'Drama,Horror,Magical-realism',
                                         'HR management':'Western,Sci-fi,Fantasy',
                                         'operating system':'Action,Kids,Educational,Inspirational',
                                         'Craft':'Adventure,Inspirational,Sci-fi,Mystery,Young-adult',
                                         'painting':'Steamy,Alternate-history,Adventure',
                                         'Fashion Design':'Young Adult,Educational,Romance',
                                         'science':'Satire,Inspirational,Action,Fantasy'})

In [8]:
# timestamp not needed
data = df.drop(['time_stamp'], axis=1)
# drop duplicate tites to make the df managable 
data_tfdif = data.drop_duplicates(['title'])
display(data.head())
display(data.shape)
display(data_tfdif.head())
display(data_tfdif.shape)

,user_id,post_id,NumLikes,title,category
0,5eece14ffc13ae660900008b,136781766,8,Sexy BANKING,"Mystery,Educational,Cyberpunk"
1,5eece14ffc13ae66090001da,136781766,5,Sexy BANKING,"Mystery,Educational,Cyberpunk"
2,5eece14efc13ae660900003a,136781766,9,Sexy BANKING,"Mystery,Educational,Cyberpunk"
3,5eece14ffc13ae6609000131,136781766,3,Sexy BANKING,"Mystery,Educational,Cyberpunk"
4,5eece14ffc13ae6609000167,136781766,2,Sexy BANKING,"Mystery,Educational,Cyberpunk"


(71800, 5)

,user_id,post_id,NumLikes,title,category
0,5eece14ffc13ae660900008b,136781766,8,Sexy BANKING,"Mystery,Educational,Cyberpunk"
18,5eece14efc13ae660900003c,43094523,10,10 Ways To Immediately Start Selling PROGRAMMING,"Fantasy,Magical-realism,Western"
37,5eece14efc13ae6609000025,42428071,4,DRAWING Adventures,"Sci-fi,Steamy,Alternate-history"
53,5eece14ffc13ae66090001d4,76472880,1,The Ultimate Guide To POLITICS,"Romance,Young-adult,Adventure,Fantasy"
67,5eece14ffc13ae66090000ac,202721843,8,ZOOLOGY And Love Have 4 Things In Common,"Kids,Western,Inspirational,Mystery"


(6000, 5)

In [9]:
combine_post_rating = data.dropna(axis = 0, subset = ['title'])

post_ratingCount = (combine_post_rating.
     groupby(by = ['title'])['NumLikes'].
     count().
     reset_index().
     rename(columns = {'NumLikes': 'TotalValue'})
     [['title', 'TotalValue']]
    )
post_ratingCount.tail()

,title,TotalValue
5995,ZOOLOGY: Do You Really Need It? This Will Help...,13
5996,ZOOLOGY: The Samurai Way,13
5997,ZOOLOGY: This Is What Professionals Do,10
5998,ZOOLOGY: What A Mistake!,5
5999,ZOOLOGY? It's Easy If You Do It Smart,14


In [10]:
rating_with_TotalValue = combine_post_rating.merge(post_ratingCount, left_on = 'title', right_on = 'title', how = 'left')
rating_with_TotalValue.head()

,user_id,post_id,NumLikes,title,category,TotalValue
0,5eece14ffc13ae660900008b,136781766,8,Sexy BANKING,"Mystery,Educational,Cyberpunk",18
1,5eece14ffc13ae66090001da,136781766,5,Sexy BANKING,"Mystery,Educational,Cyberpunk",18
2,5eece14efc13ae660900003a,136781766,9,Sexy BANKING,"Mystery,Educational,Cyberpunk",18
3,5eece14ffc13ae6609000131,136781766,3,Sexy BANKING,"Mystery,Educational,Cyberpunk",18
4,5eece14ffc13ae6609000167,136781766,2,Sexy BANKING,"Mystery,Educational,Cyberpunk",18


In [11]:
popularity_threshold = 13
rating_popular_post = rating_with_TotalValue.query('TotalValue >= @popularity_threshold')
display(rating_popular_post.tail())
display(rating_popular_post.shape)

,user_id,post_id,NumLikes,title,category,TotalValue
71697,5eece14ffc13ae66090001bb,36975738,1,I Don't Want To Spend This Much Time On MUSIC...,"Alternate-history,Adventure,Satire,Western",13
71698,5eece14ffc13ae660900012d,36975738,8,I Don't Want To Spend This Much Time On MUSIC...,"Alternate-history,Adventure,Satire,Western",13
71699,5eece14efc13ae660900006c,36975738,1,I Don't Want To Spend This Much Time On MUSIC...,"Alternate-history,Adventure,Satire,Western",13
71700,5eece14ffc13ae660900007e,36975738,2,I Don't Want To Spend This Much Time On MUSIC...,"Alternate-history,Adventure,Satire,Western",13
71701,5eece14efc13ae6609000048,36975738,8,I Don't Want To Spend This Much Time On MUSIC...,"Alternate-history,Adventure,Satire,Western",13


(38292, 6)

# FEATURE EXTRACTION via NearestNeighbors - Cosine Similarity

In [12]:
rating_popular_post = rating_popular_post.drop_duplicates(['user_id', 'title'])
rating_popular_post_pivot = rating_popular_post.pivot(index = 'title', columns = 'user_id', values = 'NumLikes').fillna(0)
rating_popular_post_matrix = csr_matrix(rating_popular_post_pivot.values)

model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(rating_popular_post_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [13]:
cosine_sim = cosine_similarity(rating_popular_post_matrix, rating_popular_post_matrix)
np.save("cosine_sim_NumLikes.npy", cosine_sim)

# Test Cosine Similarity

In [14]:
# Get a random index to predict on
query_index = np.random.choice(rating_popular_post_pivot.shape[0])
print(query_index)
distances, indices = model_knn.kneighbors(rating_popular_post_pivot.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 2)

1184


In [15]:
rating_popular_post_pivot.index[query_index]

" It's All About (The) MUSIC"

In [16]:
# Prints 5 Recommendations and their respective cosine similarity score 
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(rating_popular_post_pivot.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, rating_popular_post_pivot.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for  It's All About (The) MUSIC:

1:  You Will Thank Us - 10 Tips About BUSINESS You Need To Know, with distance of 0.6902975081185676:


# give_recommendation_cosine function

In [238]:
def give_recommendation_cosine(post_num):
    length, indices = model_knn.kneighbors(rating_popular_post_pivot.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 2)
    display(rating_popular_post_pivot.index[post_num])
    for i in range(0, len(length.flatten())):
        if i == 0:
            print('Recommendations for {0}:\n'.format(rating_popular_post_pivot.index[post_num]))
        else:
            print('{1}'.format(i, rating_popular_post_pivot.index[indices.flatten()[i]], length.flatten()[i]))

In [239]:
give_recommendation_cosine(100)

' 13 Myths About ZOOLOGY'

Recommendations for  13 Myths About ZOOLOGY:

 The Ultimate Guide To POLITICS


# FEATURE EXTRACTION via tfidf vectorizer
### Create a data-term matrix (vocabulary)

In [17]:
# Instantiate vecorizer object - call tokenize
tf_model = TfidfVectorizer(max_features=200,max_df=0.25,min_df=0.01,stop_words='english')

In [18]:
# create corpus (categories) used for the TF-IDF
corpus = data_tfdif['category']

In [19]:
tf_matrix = tf_model.fit_transform(corpus).todense()
print('The shape of the matrix is:', tf_matrix.shape)
tf_df = pd.DataFrame(tf_matrix)
tf_df.columns = sorted(tf_model.vocabulary_)
display(tf_df.head())

The shape of the matrix is: (6000, 24)


,action,adult,alternate,cyberpunk,drama,educational,fantasy,fi,fiction,historical,...,magical,mystery,realism,realistic,romance,satire,sci,steamy,western,young
0,0.0,0.000000,0.000000,0.617478,0.0,0.556202,0.000000,0.000000,0.0,0.0,...,0.000000,0.556202,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
1,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.466914,0.000000,0.0,0.0,...,0.510552,0.000000,0.510552,0.0,0.000000,0.0,0.000000,0.000000,0.510552,0.000000
2,0.0,0.000000,0.447214,0.000000,0.0,0.000000,0.000000,0.447214,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.447214,0.447214,0.000000,0.000000
3,0.0,0.495753,0.000000,0.000000,0.0,0.000000,0.453380,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.550369,0.0,0.000000,0.000000,0.000000,0.495753
4,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.495753,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.495753,0.000000


In [20]:
# Get cosine similarity of the matrix
cosine_sim = cosine_similarity(tf_matrix, tf_matrix)

In [21]:
# Fit on TF-IDF Vectors
nn = NearestNeighbors(n_neighbors=5, algorithm='ball_tree')
nn.fit(tf_matrix)

NearestNeighbors(algorithm='ball_tree')

In [22]:
# Construct a reverse mapping of indices and post titles, and drop duplicate titles if any
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

In [23]:
print(type(indices))
print(type(cosine_sim))

<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>


In [24]:
# Pickle the dtm and tf for use in the prediction
pickle.dump(tf_matrix, open('tf_matrix.pkl', 'wb'))
pickle.dump(tf_model, open('tf_model.pkl', 'wb'))
indices.to_pickle('indices.pkl')
np.save("cosine_sim.npy", cosine_sim)

# Test TF-IDF

In [37]:
# Create the test case
ideal_category = ['Mystery,Adventure,Sci-Fi,Fantasy']

new = tf_model.transform(ideal_category)
results = nn.kneighbors(new.todense())

In [38]:
# displays the cosine similarity on the top array and the index value on the bottom array
results

(array([[0.68189049, 0.68189049, 0.68189049, 0.68189049, 0.68189049]]),
 array([[170,  67, 172, 448,  21]], dtype=int64))

In [39]:
# displays the title of the first value of the first array
df['title'][results[1][0][0]]

' Here Is What You Should Do For Your PHOTOGRAPHY'

In [40]:
# displays the category of the first value of the first array 
df['category'][results[1][0][0]]

'Magical realism,Adventure,Fantasy,Action'

# give_recommendation_TF function

In [167]:
def give_recommended_TF(ideal_categories):
    new = tf_model.transform(ideal_category)
    results = nn.kneighbors(new.todense())
    display(df['title'][results[1][0][0]])

In [168]:
give_recommended_TF('Mystery,Adventure,Sci-Fi')

' 4 Ways You Can Grow Your Creativity Using MATHEMATICS'